# Clustering Toronto Neighbourhoods
## _The battle of the Neighbourhoods_

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values (OpenStreetMaps)
import requests
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


In [2]:
df = pd.read_csv('toronto_geocoded_postcodes.csv', index_col=0)
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811650,-79.195561
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785605,-79.158701
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175299
3,M1G,Scarborough,Woburn,43.768216,-79.217610
4,M1H,Scarborough,Cedarbrae,43.769608,-79.239440


In [3]:
print('Toronto summary : \n There are {} unique Postcodes (Aggregated Neighbourhoods) \
and \n {} Boroughs in the Toronto DataFrame'.format(df.Postcode.unique().size,df.Borough.unique().size))

Toronto summary : 
 There are 103 unique Postcodes (Aggregated Neighbourhoods) and 
 11 Boroughs in the Toronto DataFrame


In [4]:
df.shape

(103, 5)

In [5]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [6]:
neighborhoods = df
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.4)

# add markers to map
for lat, lng, borough, neighbourhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [7]:
#FourSquare loading
CLIENT_ID,CLIENT_SECRET,VERSION = pd.read_csv('foursquare_api_keys.csv')
print(neighborhoods.head())
print(neighborhoods.shape)

  Postcode      Borough                           Neighbourhood   Latitude  \
0      M1B  Scarborough                          Rouge, Malvern  43.811650   
1      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union  43.785605   
2      M1E  Scarborough       Guildwood, Morningside, West Hill  43.765690   
3      M1G  Scarborough                                  Woburn  43.768216   
4      M1H  Scarborough                               Cedarbrae  43.769608   

   Longitude  
0 -79.195561  
1 -79.158701  
2 -79.175299  
3 -79.217610  
4 -79.239440  
(103, 5)


In [8]:
def getBuiltUrl(neigh_lat,neigh_long,radius=1400):
    # type your answer here
    LIMIT=100
    #radius=1000
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neigh_lat, 
    neigh_long, 
    radius, 
    LIMIT)
    return url
    #url

In [9]:
# get one test Neighbourhood
neigh_post, neigh_borough, neigh_name, neigh_lat, neigh_long = df.iloc[0]
print('Latitude and longitude values of {} are {}, {}.'.format(neigh_name, 
                                                               neigh_lat, 
                                                               neigh_long))
results = requests.get(getBuiltUrl(neigh_lat,neigh_long)).json()
results

Latitude and longitude values of Rouge, Malvern are 43.811650000000036, -79.19556138899999.


{'meta': {'code': 200, 'requestId': '5c197cc24434b91ebe85a74d'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 29,
  'suggestedBounds': {'ne': {'lat': 43.82425001260005,
    'lng': -79.17813324674674},
   'sw': {'lat': 43.799049987400025, 'lng': -79.21298953125324}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c97975582b56dcb8320ebaa',
       'name': 'Canadiana exhibit',
       'location': {'lat': 43.81796218928876,
        'lng': -79.19337359666939,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.81796218928876,
          'lng': -79.1933

In [10]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# clean the json

In [11]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Canadiana exhibit,Zoo Exhibit,43.817962,-79.193374
1,African Rainforest Pavilion,Zoo Exhibit,43.817725,-79.183433
2,Lion Exhibit,Zoo Exhibit,43.819228,-79.186977
3,Orangutan Exhibit,Zoo Exhibit,43.818413,-79.182548
4,penguin exhibit,Zoo Exhibit,43.819435,-79.185959


In [12]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

29 venues were returned by Foursquare.


In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = getBuiltUrl(lat,lng,radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
#neighborhoods = neighborhoods.drop(neighborhoods.columns[0], axis=1)
neighborhoods['Neighbourhood'] = neighborhoods['Neighbourhood']+' '+neighborhoods['Postcode']


neighborhoods.head()
neighborhoods['Neighbourhood'].size

103

In [15]:

toronto_venues = getNearbyVenues(names=neighborhoods['Neighbourhood'], 
                                   latitudes=neighborhoods['Latitude'], 
                                   longitudes=neighborhoods['Longitude'], 
                                   radius=4000)

Rouge, Malvern M1B
Highland Creek, Rouge Hill, Port Union M1C
Guildwood, Morningside, West Hill M1E
Woburn M1G
Cedarbrae M1H
Scarborough Village M1J
East Birchmount Park, Ionview, Kennedy Park M1K
Clairlea, Golden Mile, Oakridge M1L
Cliffcrest, Cliffside, Scarborough Village West M1M
Birch Cliff, Cliffside West M1N
Dorset Park, Scarborough Town Centre, Wexford Heights M1P
Maryvale, Wexford M1R
Agincourt M1S
Clarks Corners, Sullivan, Tam O'Shanter M1T
Agincourt North, L'Amoreaux East, Milliken, Steeles East M1V
L'Amoreaux West, Steeles West M1W
Upper Rouge M1X
Hillcrest Village M2H
Fairview, Henry Farm, Oriole M2J
Bayview Village M2K
Silver Hills, York Mills M2L
Newtonbrook, Willowdale M2M
Willowdale South M2N
York Mills West M2P
Willowdale West M2R
Parkwoods M3A
Don Mills North M3B
Flemingdon Park, Don Mills South M3C
Bathurst Manor, Downsview North, Wilson Heights M3H
Northwood Park, York University M3J
CFB Toronto, Downsview East M3K
Downsview West M3L
Downsview Central M3M
Downsview

In [16]:
print(toronto_venues.shape)
toronto_venues.head()


(10223, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern M1B",43.81165,-79.195561,African Rainforest Pavilion,43.817725,-79.183433,Zoo Exhibit
1,"Rouge, Malvern M1B",43.81165,-79.195561,Toronto Zoo,43.820582,-79.181551,Zoo
2,"Rouge, Malvern M1B",43.81165,-79.195561,Polar Bear Exhibit,43.823372,-79.185145,Zoo
3,"Rouge, Malvern M1B",43.81165,-79.195561,Canadiana exhibit,43.817962,-79.193374,Zoo Exhibit
4,"Rouge, Malvern M1B",43.81165,-79.195561,Toronto Pan Am Sports Centre,43.790623,-79.193869,Athletics & Sports


In [17]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond M5H",100,100,100,100,100,100
Agincourt M1S,100,100,100,100,100,100
"Agincourt North, L'Amoreaux East, Milliken, Steeles East M1V",100,100,100,100,100,100
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown M9V",100,100,100,100,100,100
"Alderwood, Long Branch M8W",100,100,100,100,100,100
"Bathurst Manor, Downsview North, Wilson Heights M3H",100,100,100,100,100,100
Bayview Village M2K,100,100,100,100,100,100
"Bedford Park, Lawrence Manor East M5M",100,100,100,100,100,100
Berczy Park M5E,100,100,100,100,100,100


In [18]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
print('There are {} uniques Neighborhoods.'.format(len(toronto_venues['Neighborhood'].unique())))

There are 258 uniques categories.
There are 103 uniques Neighborhoods.


In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()
#toronto_venues['Neighborhood'].size

,Zoo Exhibit,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Liquor Store,Lounge,Malay Restaurant,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [20]:
toronto_onehot.shape

(10223, 258)

In [21]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo Exhibit,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Liquor Store,Lounge,Malay Restaurant,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"Adelaide, King, Richmond M5H",0.000000,0.01,0.000000,0.00,0.010000,0.01,0.00,0.02,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.01,0.00,0.01,0.00,0.000000,0.01,0.000000,0.00,0.00,0.000000,0.010000,0.00,0.00,0.00,0.010000,0.01,0.00,0.00,0.00,0.00,0.00,0.000000,0.01,0.000000,0.04,0.000000,0.00,0.000000,0.00,0.00,0.0

In [22]:
toronto_grouped.shape

(103, 258)

In [23]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond M5H----
                 venue  freq
0          Coffee Shop  0.07
1                Hotel  0.05
2   Italian Restaurant  0.05
3                 Café  0.04
4  Japanese Restaurant  0.03


----Agincourt M1S----
                  venue  freq
0    Chinese Restaurant  0.10
1           Coffee Shop  0.07
2          Noodle House  0.05
3  Caribbean Restaurant  0.05
4     Indian Restaurant  0.04


----Agincourt North, L'Amoreaux East, Milliken, Steeles East M1V----
                  venue  freq
0    Chinese Restaurant  0.19
1  Caribbean Restaurant  0.04
2       Bubble Tea Shop  0.04
3           Coffee Shop  0.04
4          Dessert Shop  0.03


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown M9V----
                  venue  freq
0           Coffee Shop  0.17
1  Fast Food Restaurant  0.10
2     Indian Restaurant  0.06
3        Sandwich Place  0.06
4           Pizza Place  0.06


----Alderwood, Long Branch M

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond M5H",Coffee Shop,Italian Restaurant,Hotel,Café,Japanese Restaurant,Park,Theater,Sandwich Place,Sporting Goods Shop,Diner
1,Agincourt M1S,Chinese Restaurant,Coffee Shop,Noodle House,Caribbean Restaurant,Indian Restaurant,Pharmacy,Supermarket,Sushi Restaurant,Clothing Store,Vietnamese Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Caribbean Restaurant,Coffee Shop,Bubble Tea Shop,Noodle House,Indian Restaurant,Dessert Shop,Bakery,Vietnamese Restaurant,Japanese Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Coffee Shop,Fast Food Restaurant,Pizza Place,Sandwich Place,Indian Restaurant,Bank,Grocery Store,Asian Restaurant,Caribbean Restaurant,Pharmacy
4,"Alderwood, Long Branch M8W",Coffee Shop,Bakery,Burger Joint,Café,Pizza Place,Park,Grocery Store,Restaurant,Burrito Place,Middle Eastern Restaurant
5,"Bathurst Manor, Downsview North, Wilson Height...",Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Bagel Shop,Pizza Place,Gym,Seafood Restaurant,Bookstore,Restaurant
6,Bayview Village M2K,Korean Restaurant,Coffee Shop,Supermarket,Sushi Restaurant,Chinese Restaurant,Furniture / Home Store,Burger Joint,Middle Eastern Restaurant,Bakery,Grocery Store
7,"Bedford Park, Lawrence Manor East M5M",Coffee Shop,Café,Bakery,Clothing Store,Italian Restaurant,Sushi Restaurant,Bagel Shop,Cosmetics Shop,Park,Tea Room
8,Berczy Park M5E,Coffee Shop,Hotel,Café,Theater,Gastropub,Park,Farmers Market,Thai Restaurant,Japanese Restaurant,Concert Hall
9,"Birch Cliff, Cliffside West M1N",Coffee Shop,Park,Sandwich Place,Pub,Burger Joint,Beach,Bank,Fast Food Restaurant,Pharmacy,Breakfast Spot


# Clustering

In [67]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#toronto_grouped_clustering.head()

array([2, 3, 3, 4, 6, 9, 5, 7, 2, 4], dtype=int32)

In [68]:
toronto_merged = neighborhoods#.merge(toronto_grouped['Neighborhood'], how='right', on='Neighborhood')
toronto_merged = toronto_merged.rename(columns={'Neighbourhood':'Neighborhood'})

In [69]:

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern M1B",43.811650,-79.195561,2,Zoo Exhibit,Coffee Shop,Fast Food Restaurant,Pharmacy,Pizza Place,Park,Sandwich Place,Trail,Burger Joint,Zoo
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union M1C",43.785605,-79.158701,3,Coffee Shop,Fast Food Restaurant,Park,Pizza Place,Sandwich Place,Breakfast Spot,Fried Chicken Joint,Pharmacy,Burger Joint,Beer Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill M1E",43.765690,-79.175299,3,Coffee Shop,Park,Fast Food Restaurant,Sandwich Place,Pharmacy,Pizza Place,Burger Joint,Beer Store,Supermarket,Bank
3,M1G,Scarborough,Woburn M1G,43.768216,-79.217610,4,Coffee Shop,Indian Restaurant,Gym,Caribbean Restaurant,Pizza Place,Park,Clothing Store,Pharmacy,Sandwich Place,Liquor Store
4,M1H,Scarborough,Cedarbrae M1H,43.769608,-79.239440,6,Coffee Shop,Indian Restaurant,Chinese Restaurant,Gym,Pharmacy,Bakery,Caribbean Restaurant,Clothing Store,Bookstore,Supermarket


In [70]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine clusters

## cluster 0

In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,East York,0,Café,Greek Restaurant,Park,Bakery,Coffee Shop,Brewery,Gastropub,Pizza Place,Indian Restaurant,Cocktail Bar
85,Queen's Park,0,Café,Hotel,Italian Restaurant,Restaurant,Coffee Shop,Sandwich Place,Vegetarian / Vegan Restaurant,Theater,Japanese Restaurant,Concert Hall
87,East Toronto,0,Coffee Shop,Hotel,Café,Italian Restaurant,Restaurant,Sandwich Place,Park,Theater,Farmers Market,Steakhouse
88,Etobicoke,0,Park,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Bakery,Burger Joint,Liquor Store,Pizza Place,BBQ Joint
89,Etobicoke,0,Coffee Shop,Bakery,Burger Joint,Café,Pizza Place,Park,Grocery Store,Restaurant,Burrito Place,Middle Eastern Restaurant
92,Etobicoke,0,Coffee Shop,Burger Joint,Sushi Restaurant,Furniture / Home Store,Pizza Place,Bakery,Dessert Shop,Grocery Store,Seafood Restaurant,Burrito Place
100,Etobicoke,0,Coffee Shop,Pharmacy,Sandwich Place,Pizza Place,Hotel,Bank,Steakhouse,American Restaurant,Golf Course,Beer Store
101,Etobicoke,0,Coffee Shop,Fast Food Restaurant,Pizza Place,Sandwich Place,Indian Restaurant,Bank,Grocery Store,Asian Restaurant,Caribbean Restaurant,Pharmacy


## cluster 1

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,1,Middle Eastern Restaurant,Coffee Shop,Burger Joint,Chinese Restaurant,Supermarket,Indian Restaurant,Mediterranean Restaurant,Pet Store,Pharmacy,Falafel Restaurant
16,Scarborough,1,Zoo Exhibit,Coffee Shop,Fast Food Restaurant,Sandwich Place,Grocery Store,Pharmacy,Pizza Place,Hakka Restaurant,Restaurant,Burger Joint
21,North York,1,Korean Restaurant,Coffee Shop,Chinese Restaurant,Sushi Restaurant,Middle Eastern Restaurant,Supermarket,Café,Grocery Store,Seafood Restaurant,Shopping Mall
30,North York,1,Clothing Store,Coffee Shop,Bakery,Deli / Bodega,Vietnamese Restaurant,Athletics & Sports,Boutique,Café,Restaurant,Sporting Goods Shop
35,East York,1,Bakery,Gastropub,Middle Eastern Restaurant,Café,Coffee Shop,Brewery,Thai Restaurant,Gym / Fitness Center,Indian Restaurant,Turkish Restaurant
48,Central Toronto,1,Park,Italian Restaurant,Café,Coffee Shop,Indian Restaurant,Tea Room,Grocery Store,Spa,Dessert Shop,Gym
51,Downtown Toronto,1,Coffee Shop,Park,Café,Farmers Market,Japanese Restaurant,Breakfast Spot,Pizza Place,Diner,Gastropub,Hotel
57,Downtown Toronto,1,Coffee Shop,Hotel,Café,Steakhouse,Concert Hall,Theater,Restaurant,Sandwich Place,Gastropub,Art Gallery
65,Central Toronto,1,Café,Coffee Shop,Bar,Cocktail Bar,Sandwich Place,Vegetarian / Vegan Restaurant,Grocery Store,Italian Restaurant,Pizza Place,Dessert Shop
73,York,1,Café,Italian Restaurant,Coffee Shop,BBQ Joint,Spa,Bar,Park,Bakery,Ice Cream Shop,Indian Restaurant


# Cluster 1

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,1,Middle Eastern Restaurant,Coffee Shop,Burger Joint,Chinese Restaurant,Supermarket,Indian Restaurant,Mediterranean Restaurant,Pet Store,Pharmacy,Falafel Restaurant
16,Scarborough,1,Zoo Exhibit,Coffee Shop,Fast Food Restaurant,Sandwich Place,Grocery Store,Pharmacy,Pizza Place,Hakka Restaurant,Restaurant,Burger Joint
21,North York,1,Korean Restaurant,Coffee Shop,Chinese Restaurant,Sushi Restaurant,Middle Eastern Restaurant,Supermarket,Café,Grocery Store,Seafood Restaurant,Shopping Mall
30,North York,1,Clothing Store,Coffee Shop,Bakery,Deli / Bodega,Vietnamese Restaurant,Athletics & Sports,Boutique,Café,Restaurant,Sporting Goods Shop
35,East York,1,Bakery,Gastropub,Middle Eastern Restaurant,Café,Coffee Shop,Brewery,Thai Restaurant,Gym / Fitness Center,Indian Restaurant,Turkish Restaurant
48,Central Toronto,1,Park,Italian Restaurant,Café,Coffee Shop,Indian Restaurant,Tea Room,Grocery Store,Spa,Dessert Shop,Gym
51,Downtown Toronto,1,Coffee Shop,Park,Café,Farmers Market,Japanese Restaurant,Breakfast Spot,Pizza Place,Diner,Gastropub,Hotel
57,Downtown Toronto,1,Coffee Shop,Hotel,Café,Steakhouse,Concert Hall,Theater,Restaurant,Sandwich Place,Gastropub,Art Gallery
65,Central Toronto,1,Café,Coffee Shop,Bar,Cocktail Bar,Sandwich Place,Vegetarian / Vegan Restaurant,Grocery Store,Italian Restaurant,Pizza Place,Dessert Shop
73,York,1,Café,Italian Restaurant,Coffee Shop,BBQ Joint,Spa,Bar,Park,Bakery,Ice Cream Shop,Indian Restaurant


## cluster 2

In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,2,Zoo Exhibit,Coffee Shop,Fast Food Restaurant,Pharmacy,Pizza Place,Park,Sandwich Place,Trail,Burger Joint,Zoo
8,Scarborough,2,Coffee Shop,Sandwich Place,Park,Fast Food Restaurant,Pharmacy,Bank,Pizza Place,Chinese Restaurant,Gym,Grocery Store
12,Scarborough,2,Chinese Restaurant,Coffee Shop,Noodle House,Caribbean Restaurant,Indian Restaurant,Pharmacy,Supermarket,Sushi Restaurant,Clothing Store,Vietnamese Restaurant
14,Scarborough,2,Chinese Restaurant,Caribbean Restaurant,Coffee Shop,Bubble Tea Shop,Noodle House,Indian Restaurant,Dessert Shop,Bakery,Vietnamese Restaurant,Japanese Restaurant
15,Scarborough,2,Chinese Restaurant,Bakery,Caribbean Restaurant,Japanese Restaurant,Bubble Tea Shop,Sushi Restaurant,Asian Restaurant,Tea Room,Noodle House,Vietnamese Restaurant
17,North York,2,Chinese Restaurant,Japanese Restaurant,Bakery,Coffee Shop,Caribbean Restaurant,Asian Restaurant,Supermarket,Fish Market,Sporting Goods Shop,Middle Eastern Restaurant
19,North York,2,Korean Restaurant,Coffee Shop,Supermarket,Sushi Restaurant,Chinese Restaurant,Furniture / Home Store,Burger Joint,Middle Eastern Restaurant,Bakery,Grocery Store
20,North York,2,Bakery,Japanese Restaurant,Burger Joint,Café,Coffee Shop,Bubble Tea Shop,Shopping Mall,Middle Eastern Restaurant,Seafood Restaurant,Steakhouse
22,North York,2,Korean Restaurant,Coffee Shop,Café,Burger Joint,Sushi Restaurant,Middle Eastern Restaurant,Grocery Store,Bakery,Steakhouse,Burrito Place
27,North York,2,Bakery,Middle Eastern Restaurant,Japanese Restaurant,Park,Supermarket,Grocery Store,Gym / Fitness Center,Indian Restaurant,Restaurant,Coffee Shop


## cluster 3

In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,3,Coffee Shop,Fast Food Restaurant,Park,Pizza Place,Sandwich Place,Breakfast Spot,Fried Chicken Joint,Pharmacy,Burger Joint,Beer Store
2,Scarborough,3,Coffee Shop,Park,Fast Food Restaurant,Sandwich Place,Pharmacy,Pizza Place,Burger Joint,Beer Store,Supermarket,Bank
18,North York,3,Chinese Restaurant,Bakery,Coffee Shop,Caribbean Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Seafood Restaurant,Supermarket,Burger Joint,Italian Restaurant
24,North York,3,Coffee Shop,Korean Restaurant,Middle Eastern Restaurant,Café,Sushi Restaurant,Grocery Store,Pizza Place,Restaurant,Ramen Restaurant,Sandwich Place
34,North York,3,Middle Eastern Restaurant,Burger Joint,Coffee Shop,Supermarket,Park,Bakery,Gym / Fitness Center,Mediterranean Restaurant,Indian Restaurant,Movie Theater
42,East Toronto,3,Café,Park,Beach,Coffee Shop,Brewery,Breakfast Spot,Bakery,Gastropub,Pizza Place,Ice Cream Shop
53,Downtown Toronto,3,Coffee Shop,Café,Japanese Restaurant,Hotel,Gastropub,Park,Breakfast Spot,Diner,Farmers Market,Steakhouse
61,Downtown Toronto,3,Coffee Shop,Hotel,Café,Japanese Restaurant,Farmers Market,Sandwich Place,Theater,Park,Pizza Place,Diner


## cluster 4

In [60]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,4,Coffee Shop,Indian Restaurant,Gym,Caribbean Restaurant,Pizza Place,Park,Clothing Store,Pharmacy,Sandwich Place,Liquor Store
9,Scarborough,4,Coffee Shop,Park,Sandwich Place,Pub,Burger Joint,Beach,Bank,Fast Food Restaurant,Pharmacy,Breakfast Spot
25,North York,4,Middle Eastern Restaurant,Coffee Shop,Japanese Restaurant,Italian Restaurant,Supermarket,Park,Gym / Fitness Center,Mediterranean Restaurant,Café,Burger Joint
37,East Toronto,4,Park,Café,Coffee Shop,Beach,Breakfast Spot,Bakery,Brewery,Gastropub,Pub,Cocktail Bar
38,East York,4,Park,Café,Bakery,Coffee Shop,Italian Restaurant,Restaurant,Grocery Store,Sushi Restaurant,Supermarket,Middle Eastern Restaurant
41,East Toronto,4,Park,Coffee Shop,Café,Greek Restaurant,Japanese Restaurant,Pizza Place,Diner,Bakery,Italian Restaurant,Thai Restaurant
47,Central Toronto,4,Italian Restaurant,Park,Café,Coffee Shop,Bakery,Trail,Supermarket,Tea Room,Indian Restaurant,BBQ Joint
52,Downtown Toronto,4,Coffee Shop,Café,Japanese Restaurant,Italian Restaurant,Thai Restaurant,Restaurant,Farmers Market,Theater,Gastropub,Hotel
56,Downtown Toronto,4,Coffee Shop,Hotel,Café,Theater,Gastropub,Park,Farmers Market,Thai Restaurant,Japanese Restaurant,Concert Hall
59,Downtown Toronto,4,Park,Italian Restaurant,Coffee Shop,Bakery,Gym,Hotel,Pizza Place,French Restaurant,Theater,Sporting Goods Shop


## cluster 5


In [61]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,5,Coffee Shop,Middle Eastern Restaurant,Burger Joint,Park,Indian Restaurant,Chinese Restaurant,Pharmacy,Supermarket,Burrito Place,Breakfast Spot
69,Downtown Toronto,5,Coffee Shop,Hotel,Café,Italian Restaurant,Restaurant,Sandwich Place,Park,Theater,Farmers Market,Steakhouse
97,North York,5,Fast Food Restaurant,Coffee Shop,Pizza Place,Pharmacy,Bank,Sandwich Place,Fried Chicken Joint,Beer Store,Vietnamese Restaurant,Asian Restaurant
98,York,5,Coffee Shop,Sandwich Place,Pizza Place,Fast Food Restaurant,Bank,Pharmacy,Grocery Store,Golf Course,Vietnamese Restaurant,Bakery


## cluster 6

In [62]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Scarborough,6,Coffee Shop,Indian Restaurant,Chinese Restaurant,Gym,Pharmacy,Bakery,Caribbean Restaurant,Clothing Store,Bookstore,Supermarket
10,Scarborough,6,Coffee Shop,Chinese Restaurant,Middle Eastern Restaurant,Clothing Store,Indian Restaurant,Supermarket,Grocery Store,Breakfast Spot,Gym,Caribbean Restaurant
23,North York,6,Coffee Shop,Park,Café,Bagel Shop,Sushi Restaurant,Bakery,Tea Room,Burger Joint,Steakhouse,Bridal Shop
26,North York,6,Middle Eastern Restaurant,Bakery,Burger Joint,Coffee Shop,Japanese Restaurant,Supermarket,Park,Furniture / Home Store,Shopping Mall,Gym / Fitness Center
33,North York,6,Coffee Shop,Sandwich Place,Pizza Place,Bank,Fast Food Restaurant,Hotel,Pharmacy,Restaurant,Thai Restaurant,Flea Market
39,East York,6,Park,Café,Greek Restaurant,Bakery,Grocery Store,Gastropub,Brewery,BBQ Joint,Ice Cream Shop,Indian Restaurant
44,Central Toronto,6,Café,Bakery,Coffee Shop,Italian Restaurant,Park,Restaurant,Gym,Ice Cream Shop,Sandwich Place,Asian Restaurant
60,Downtown Toronto,6,Coffee Shop,Hotel,Italian Restaurant,Café,Restaurant,Japanese Restaurant,Sandwich Place,Park,Theater,Farmers Market
67,Downtown Toronto,6,Café,Sandwich Place,Italian Restaurant,Bakery,Coffee Shop,Theater,Restaurant,Pizza Place,Cocktail Bar,Bar
74,York,6,Café,Italian Restaurant,Coffee Shop,Bar,Bakery,Ice Cream Shop,BBQ Joint,Brewery,Indian Restaurant,Gastropub


## cluster 7

In [63]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Scarborough,7,Middle Eastern Restaurant,Burger Joint,Coffee Shop,Breakfast Spot,Gym / Fitness Center,Supermarket,Park,Bank,Fish & Chips Shop,Indian Restaurant
28,North York,7,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Bagel Shop,Pizza Place,Gym,Seafood Restaurant,Bookstore,Restaurant
29,North York,7,Coffee Shop,Pharmacy,Sandwich Place,Grocery Store,Pizza Place,Restaurant,Sushi Restaurant,Middle Eastern Restaurant,Discount Store,Park
31,North York,7,Coffee Shop,Pizza Place,Vietnamese Restaurant,Sandwich Place,Fast Food Restaurant,Pharmacy,Bank,Grocery Store,Tennis Stadium,Hotel
45,Central Toronto,7,Coffee Shop,Café,Bakery,Park,Sushi Restaurant,Italian Restaurant,Burger Joint,Gastropub,Gym,Diner
54,Downtown Toronto,7,Coffee Shop,Hotel,Café,Japanese Restaurant,Restaurant,Theater,Farmers Market,Thai Restaurant,Diner,Gastropub
55,Downtown Toronto,7,Coffee Shop,Hotel,Japanese Restaurant,Café,Gastropub,Theater,Park,Restaurant,Farmers Market,Art Gallery
58,Downtown Toronto,7,Coffee Shop,Italian Restaurant,Hotel,Café,Japanese Restaurant,Park,Theater,Sandwich Place,Sporting Goods Shop,Diner
63,Central Toronto,7,Coffee Shop,Café,Italian Restaurant,Bakery,Park,Clothing Store,Burger Joint,Restaurant,Cosmetics Shop,Breakfast Spot
64,Central Toronto,7,Italian Restaurant,Café,Coffee Shop,Park,Middle Eastern Restaurant,Ice Cream Shop,Bakery,BBQ Joint,Indian Restaurant,Tea Room


## cluster 8

In [64]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 8, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
78,West Toronto,8,Café,Bar,Pizza Place,Bakery,Italian Restaurant,Asian Restaurant,Sandwich Place,American Restaurant,Cocktail Bar,Dessert Shop
93,Etobicoke,8,Coffee Shop,Bakery,Burger Joint,Pharmacy,Italian Restaurant,Café,Grocery Store,Pub,Liquor Store,Pizza Place


## cluster 9

In [65]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 9, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,9,Coffee Shop,Pharmacy,Sandwich Place,Indian Restaurant,Pizza Place,Fast Food Restaurant,Bank,Chinese Restaurant,Park,Beer Store
13,Scarborough,9,Chinese Restaurant,Coffee Shop,Caribbean Restaurant,Breakfast Spot,Noodle House,Vietnamese Restaurant,Bakery,Fast Food Restaurant,Greek Restaurant,Bookstore
36,East York,9,Park,Coffee Shop,Bakery,Café,Gastropub,Beach,Brewery,Pizza Place,Breakfast Spot,Greek Restaurant
46,Central Toronto,9,Coffee Shop,Bakery,Café,Italian Restaurant,Sushi Restaurant,Park,Burger Joint,Dessert Shop,Diner,Wings Joint
62,North York,9,Coffee Shop,Café,Bakery,Clothing Store,Italian Restaurant,Sushi Restaurant,Bagel Shop,Cosmetics Shop,Park,Tea Room
66,Downtown Toronto,9,Café,Sandwich Place,Italian Restaurant,Vegetarian / Vegan Restaurant,Cocktail Bar,Coffee Shop,Pizza Place,Park,Dessert Shop,Art Gallery


# Thank you very much!